## Семинар 11

Всем привет и добро пожаловать на наш семинар по сбору новостей. Сегодня мы ставим себе следующие цели:

1. Собрать все заголовки статей на определенную тему, отфильтровав по дате с сайта делового издания "Коммерсант"
2. Узнать про хитрости, которые могут помочь в сборе данных
3. Провести самостоятельную практику с похожими заданиями

Для этого мы воспользуемся уже знакомыми Вам библиотеками requests и BeautifulSoup, а также познакомимся с новой – Selenium.


<img src="https://avatars.mds.yandex.net/get-zen_doc/3892121/pub_5f7ab7fb8d3ae5589bb054aa_5f7ab85061e6d41ef5615d94/scale_1200" width=700>

## Забираем заголовки с Коммерсанта

In [6]:
import requests 
from bs4 import BeautifulSoup

In [7]:
from fake_useragent import UserAgent

Итак, начнем с простого – проверим, соберется ли у нас информация с главной страницы Коммерсанта или нужно искать специальные примочки.

In [9]:
url = 'http://www.kommersant.ru/'
import requests

headers = {
    'User-Agent': 'UserAgent().chrome',
}

response = requests.get(url, headers=headers)

ConnectionError: HTTPConnectionPool(host='www.kommersant.ru', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D10DBFF370>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

Сделаем деревце, по которому можно искать нужные элементы с помощью `bs4`:

In [ ]:
tree = BeautifulSoup(response.content, 'html.parser')

Найдем главную новость в тексте.

In [ ]:
tree.find_all('a', {'class': 'top_news_main__link link'})

Достанем только текст:

In [ ]:
tree.find('a', {'class': 'top_news_main__link link'}).text.strip()

### Добавляем селениум :)

Библиотека `selenium` – набор инструментов для интерактивной работы в браузере средствами Python. Вообще Selenium ‒ это целый проект, в котором есть разные инструменты. Мы рассмотрим один из самых распространенных ‒ Selenium WebDriver, модуль, который позволяется Python встраиваться в браузер и работать в нем как пользователь: кликать на ссылки и кнопки, заполнять формы, выбирать опции в меню и прочее. 

In [ ]:
# через восклицательный знак обращемся к командной строке (на маке называется terminal)
# pip – менеджер пакетов для питона, который позволяет нам поставить библиотеку
!pip install selenium
!pip install webdriver-manager

Для того, чтобы воспользоваться библиотекой, нужно загрузить вебдрайвер для Вашего браузера. Подробнее можно почитать [в пункте 1.5 документации про установку](https://selenium-python.readthedocs.io/installation.html). План действий такой: качате драйвер – прописываете путь в переменной PATH – используете.

Но мы воспользуемся лайфхаком, чтобы не мучиться долго с установкой. Это библиотека `webdriver-manager`, которая скачает вебдрайвер за Вас. Подробнее в [документации](https://pypi.org/project/webdriver-manager/) (там же можно посмотреть код для других браузеров).

In [ ]:
from selenium.webdriver.common.keys import Keys

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

На Вашем компьютере откроется пустая страничка. Давайте перейдем на сайт Коммерсанта.

In [ ]:
driver.get('https://www.kommersant.ru/')

Давайте найдем главный заголовок еще одним способом. Сначала найдем элемент, помня имя класса (см. скрины выше), потом достанем его html код.

In [ ]:
main_news = driver.find_element_by_class_name("top_news_main__name")
main_news

In [ ]:
main_news.get_attribute('innerHTML')

In [ ]:
small_tree = BeautifulSoup(main_news.get_attribute('innerHTML'), 'html.parser')

In [ ]:
small_tree

In [ ]:
small_tree.text.strip()

Ура, получили заголовок главной новости. Если он отличается от того, что на скрине, то нестрашно – новостные сайты быстро меняют статьи на главных страницах. Остальное можно достать по аналогии :)

Перейдем к более интересному – соберем все новости на определенную тему и срок.

Предлагаю попробовать собрать все новости, содержащие фразу "центральный банк" за период с 24 августа 2021 по текущий день. То есть, переводя это на программистский, нам нужно проделать следующие действия:

1. Найти окно поиска, кликнуть
2. Ввести в него ключевую фразу, нажать кнопку поиска
3. Нажать кнопку расширенный поиск
4. Найти кнопку, где изменяем дату начала поиска, выставить нужную нам
5. Собрать информацию

Давайте начнем :) В прошлый раз мы воспользовались поиском с помощью селектора `.find_element_by_css_selector()`. Теперь добавим немного разнообразия и сделаем поиском через XPath. Получить ее можно по старой схеме: наводим мышь на окно поиска – кликаем посмотреть код – правой кнопкой кликаем по мыши на выделенном коде – выбираем copy – copy xpath.   

In [ ]:
# найденный по инструкции выше xpath к лупе
xpath_query = '//*[@id="js-navsearch-submit"]'
# находим окно поиска
search = driver.find_element_by_xpath(xpath_query)
# кликаем на него
search.click()

In [ ]:
# найденный по инструкции выше xpath к окну поиска
xpath_query = '//*[@id="js-navsearch-query"]'
# находим окно поиска
search = driver.find_element_by_xpath(xpath_query)
# кликаем на него
search.click()

In [ ]:
search_term = "центральный банк"
# печатаем фразу для поиска в окне для поиска
search.send_keys(search_term)

In [ ]:
# нажимаем кнопку enter
search.send_keys(Keys.RETURN)

Дальше мы все уже знаем. Откройте в соседней с ноутбуком вкладке сайт коммерсанта и доставайте оттуда нужные селекторы / xpath (неважно).

In [ ]:
# находим селектор для кнопки расширенный поиск и нажимаем ее
selector2 = "body > main > div > div > section > div.grid-col.grid-col-s3 > form > div.ui-field_pack > label"


In [ ]:
ext_search = driver.find_element_by_css_selector(selector2)

In [ ]:
ext_search.click()

In [ ]:
# находим селектор для поля ввода даты
selector3 = "body > main > div > div > section > div.grid-col.grid-col-s3 > form > div.ui-collapse.js-toggle-collapse.js-toggle-item.ui-collapse--show > section.simple_page_section.simple_page_section--form.js-search-settings > div.grid.ui-collapse.js-toggle-collapse.js-toggle-item.ui-collapse--show > div:nth-child(1) > div > input"


In [ ]:
date = driver.find_element_by_css_selector(selector3)

Обратите внимание на картинку ниже – дата начала поиска по дефолту вбита в окошко, надо ее удалить.

<img src="imgs/pic9.png" width=500>

In [ ]:
# удаляем введеный по дефолту текст в ячейке
date.send_keys(Keys.SHIFT, Keys.END, Keys.BACK_SPACE)

In [ ]:
# вводим нужную дату и надижимаем enter
date_start = "24.08.2021"
date.send_keys(date_start)

In [ ]:
date.send_keys(Keys.RETURN)

In [ ]:
#driver.close()

Ура, получили нужную выдачу! Попробуем перейти на следующую страничку.

In [ ]:
# путь к кнопке следующая страница
xpath3 = "/html/body/main/div/div/section/div[1]/div[3]/a"

In [ ]:
second_page = driver.find_element_by_xpath(xpath3)

In [ ]:
second_page.click()

Посмотрим на адрес нашей странички:

In [ ]:
driver.current_url

In [ ]:
# driver.page_source

Обратите внимание на параметр `page=2`. Если мы будем менять номера, то будем перемещаться по всем страницам с заголовками, удовлетворяющим нашим условиям. Осталось написать функцию, которая будет доставать нужную информацию с одной странички, и запустить ее циклом для всех.

Начнем с того, как задать url. Обратите внимание на обратный слэш, это так называемый line continuation character. Он означает, что код продолжится на следующей строке. Также обратите внимание на букву f перед продолжением url-адреса на 3 строчке – она позваоляет мне подставить значение переменной `{page_num}` в середину строки.

In [ ]:
page_num = 1
url = 'https://www.kommersant.ru/search/results?search_query='\
    '%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9+%D0%B1'\
    '%D0%B0%D0%BD%D0%BA&sort_type=1&search_full=1&time_range=2&'\
    f'dateStart=2021-08-24&dateEnd=2021-10-15&page={page_num}'

In [ ]:
url

Как обычно забираем HTML-разметку и делаем деревце бьютифул супом.

In [ ]:
response2 = requests.get(url)
response2

In [ ]:
tree_search = BeautifulSoup(response2.content, 'html.parser')

Уже знакомый  механизм поиска элемента по html разметке.

In [ ]:
# находим заголовки
headers = tree_search.find_all('h2', {'class': 'uho__name rubric_lenta__item_name'})

In [ ]:
len(headers)

In [ ]:
headers[0]

Если достать из тега текст, то можно заметить, что есть пробелы / переходы на новые строки в начале и конце. Метод `.strip()` избавится от них.

In [ ]:
headers[0].text

In [ ]:
headers[0].text.strip()

In [ ]:
# находим подзаголовки
subheaders = tree_search.find_all('h3', \
                {'class': 'uho__subtitle rubric_lenta__item_subtitle'})

In [ ]:
len(subheaders)

Подзаголовки есть не у всех новостей!

In [ ]:
subheaders[0]

In [ ]:
subheaders[0].text

In [ ]:
subheaders[0].text.strip()

In [ ]:
# напишем функцию, которая будет выдавать список из словарей 
# в каждом словаре заголовок и описание
def get_page_info(page_num):
    url = 'https://www.kommersant.ru/search/results?search_query='\
        '%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9+%D0%B1'\
        '%D0%B0%D0%BD%D0%BA&sort_type=1&search_full=1&time_range=2&'\
        f'dateStart=2021-08-24&dateEnd=2021-10-15&page={page_num}'
    
    response = requests.get(url)
    tree_search = BeautifulSoup(response.content, 'html.parser')
    headers = tree_search.find_all('h2', \
                                   {'class': 'uho__name rubric_lenta__item_name'})
    subheaders = tree_search.find_all('h3', \
                            {'class': 'uho__subtitle rubric_lenta__item_subtitle'})
    result = []
    for i in range(len(headers)):
        header = headers[i]
        try:
            subheader = subheaders[i]
            d = {'article_header': header.text.strip(),
                 'article_subheader': subheader.text.strip()}
        except:
            d = {'article_header': header.text.strip(),
                 'article_subheader': ''}
        result.append(d)
    return result

In [ ]:
all_data = []
for n in range(1, 4):
    all_data.extend(get_page_info(n))

In [ ]:
len(all_data)

Пока не очень знакомая библиотека сделает табличку из списка и позволит сохранить ее в файл.

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(all_data)

In [ ]:
df.head()

In [ ]:
# сохранить в csv формат
# index=False сделает так, чтобы колонка с индексами не вогла в итоговый файл
df.to_csv('all_data.csv', index=False)

In [ ]:
# сохранить в xlsx формат
df.to_excel('all_data.xlsx', index=False)

In [ ]:
# не забываем закрыть браузер драйвером после завершения работы :)
driver.close()

## Практика

Основная цель практики – убедиться, что Вам понятен код с семинара. Попробуйте собрать информацию по аналогии с сайта Коммерсанта. Фразу для поиска оставляем на Ваш выбор, срок – за последний месяц.